In [1]:
import gym

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import sklearn
from sklearn.metrics import mean_squared_error, r2_score

import pickle

import os



In [2]:
torch.cuda.empty_cache()

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"
print(device)

cuda


## Generate samples from PETS models

In [10]:
class EnsembleModel(nn.Module):
    def __init__(self, input_size, output_size, num_layers, num_nodes, activation, num_ensembles):
        super(EnsembleModel, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.num_nodes = num_nodes
        self.activation = activation
        self.num_ensembles = num_ensembles

        self.ensemble_models = nn.ModuleList()
        for _ in range(num_ensembles):
            model = self._build_model()
            self.ensemble_models.append(model)

    def _build_model(self):
        layers = []
        layers.append(nn.Linear(self.input_size, self.num_nodes))
        layers.append(self.activation)

        for _ in range(self.num_layers - 1):
            layers.append(nn.Linear(self.num_nodes, self.num_nodes))
            layers.append(self.activation)

        layers.append(nn.Linear(self.num_nodes, self.output_size))
        return nn.Sequential(*layers)

    def forward(self, x):
        means = []
        stds = []
        for model in self.ensemble_models:
            output = model(x)
            mean = output[:, :self.output_size // 2]
            std = torch.exp(output[:, self.output_size // 2:])
            means.append(mean)
            stds.append(std)
        means = torch.stack(means, dim=0)
        stds = torch.stack(stds, dim=0)
        final_mean = torch.mean(means, dim=0)
        final_std = torch.mean(stds, dim=0)
        return final_mean, final_std
    
# Loss function
def gaussian_likelihood(mean_pred, std_pred, target):
    EPS = 1e-6 
    std_pred = torch.clamp(std_pred, min=EPS) 
    
    # Negative log likelihood of Gaussian distribution
    loss = torch.mean(0.5 * ((target - mean_pred) / std_pred) ** 2 + torch.log(std_pred) + 0.5 * np.log(2 * np.pi))
    
    if torch.isnan(loss):
#         print('target: ',target,' mean: ', mean_pred, ' std: ',std_pred)
        print('mean: ', mean_pred, ' std: ',std_pred)
    return loss

In [11]:
class Sample:
    def __init__(self, model, env):
        self.model = model
        # Create the CartPole environment
        self.env = env
        self.df_discrete = None
        self.df_MDP = None
        
    def generateSample(self, filename, num_samples, max_episode_length=200, noise = 0):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#         device = "cpu"
        self.model.to(device)
        state_size = self.env.observation_space.shape[0]
        #state_size = 4
        samples = []
        sample_count = 0
        while sample_count < num_samples:
            state = self.env.reset()[0]
            episode_length = 0

            while episode_length <= max_episode_length:
                action = self.env.action_space.sample()
                input_model = list(state) + [action]
                input_model = torch.tensor(input_model, device=device, dtype=torch.float32)
                input_model = input_model.unsqueeze(0)
                print(input_model)

                #print(f"Model is on device: " + str(next(model.parameters()).device))
                #print(f"input_model is on device: {input_model.device}")
                mean_next_state, std_next_state = self.model(input_model)
                print("Next State : mean, std")
                print(mean_next_state, std_next_state)
            
                
                # Apply softplus to std_next_state to ensure it is positive
                std_next_state = torch.nn.functional.softplus(std_next_state)
        
                std_next_state = torch.clamp(std_next_state, min = 0, max=1e+2)

                # Create a normal distribution with the given mean and std
                normal_dist = torch.distributions.Normal(mean_next_state, std_next_state)

                # Sample from the distribution
                next_state = normal_dist.sample()[0]
                print("Next State")
                print(next_state)
                
                #print(next_state)
                state = list(state)
                action = [str(action)]
                next_state = next_state.tolist()
                next_state, reward, done = next_state[:-2], next_state[-2], next_state[-1]
                reward = [reward]
                #next_state = [noise*random.uniform(-1, 1)+next_state[i] for i in range(state_size)]
                #next_state = self.add_noise(next_state)

                sample = state + action + reward + next_state
                samples.append(sample)
                state = next_state
                
                sample_count = sample_count + 1
                episode_length += 1
                if done > 0.5 or done < -0.5:
                    break
#                 break
#             break
                    
        df = pd.DataFrame(samples)
        df.columns = [f'variable_{i}' for i in range(state_size)] + ['action','reward'] + [f'nx_variable_{i}' for i in range(state_size)]
        self.df_data = df
        df.to_csv(filename, index=False)  

In [ ]:
env_names = ['CartPole-v1', 'MountainCarContinuous-v0', 'MountainCar-v0', 'Pendulum-v1']
num_samples_list = ['10k', '20k', '30k', '40k', '50k']

for env_name in env_names:

    env = gym.make(env_name)
    batch_size = 32


    for num_samples in num_samples_list:
        
        #Initialize parameters for model
        action = env.action_space.sample()

        if isinstance(action, int):
            input_size = env.observation_space.shape[0] + 1
        elif isinstance(action, np.ndarray):
            input_size = env.observation_space.shape[0] + len(action)

        output_size = 2*(env.observation_space.shape[0] + 2)
        num_layers = 3
        num_nodes = 20
        activation = nn.ReLU()
        num_ensembles = 5
        learning_rate = 0.01

        model = EnsembleModel(input_size, output_size, num_layers, num_nodes, activation, num_ensembles).to(device)
        
        
        model_weights_file_path = 'pets_'+ env_name + '_' + num_samples +'.pth' 
#         model_weights_file_path = 'pets_'+ env_name + '_' + num_samples + '_g=5' +'.pth' 
        model.load_state_dict(torch.load(model_weights_file_path))
        
        sample = Sample(model,env)
        sample_filename = 'pets_'+ env_name + '_' + num_samples + '_sample.csv'
#         sample_filename = 'pets_'+ env_name + '_' + num_samples + '_g=5_sample.csv'
        sample.generateSample(num_samples = 10000, noise = 0, filename = sample_filename)
        
#         break
#     break
    

# Monte Carlo

In [3]:
class MC_Dropout_Net(nn.Module):
    def __init__(self, input_size, output_size, num_hidden_layers, hidden_layer_nodes, activation, dropout_prob, num_network):
        super(MC_Dropout_Net, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.num_hidden_layers = num_hidden_layers
        self.hidden_layer_nodes = hidden_layer_nodes
        self.activation = activation
        self.num_network = num_network
        
        # Define the layers
        self.input_layer = nn.Linear(input_size, hidden_layer_nodes)
        self.hidden_layers = nn.ModuleList()
        for _ in range(num_hidden_layers):
            self.hidden_layers.append(nn.Linear(hidden_layer_nodes, hidden_layer_nodes))
            self.hidden_layers.append(nn.Dropout(p=dropout_prob))
        self.output_layer = nn.Linear(hidden_layer_nodes, output_size)

    def forward(self, x):
        total_output = 0.0
        for i in range(self.num_network):
            x_temp = x
            x_temp = self.activation(self.input_layer(x_temp))
            for hidden_layer in self.hidden_layers:
                x_temp = self.activation(hidden_layer(x_temp))
            output = self.output_layer(x_temp)
            total_output += output
        average_output = total_output / self.num_network
        return average_output


In [8]:
class Sample:
    def __init__(self, model, env):
        self.model = model
        # Create the CartPole environment
        self.env = env
        self.df_discrete = None
        self.df_MDP = None
        
    def generateSample(self, filename, num_samples, max_episode_length=200, noise = 0):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        state_size = self.env.observation_space.shape[0]
        #state_size = 4
        samples = []
        sample_count = 0
        while sample_count < num_samples:
            state = self.env.reset()[0]
            episode_length = 0

            while episode_length <= max_episode_length:
                action = self.env.action_space.sample()
                input_model = list(state) + [action]
                input_model = torch.tensor(input_model, device=device, dtype=torch.float32)
                #print(f"Model is on device: " + str(next(model.parameters()).device))
                #print(f"input_model is on device: {input_model.device}")
                next_state = self.model(input_model)
                state = list(state)
                action = [str(action)]
                next_state = next_state.tolist()
                next_state, reward, done = next_state[:-2], next_state[-2], next_state[-1]
                reward = [reward]
                #next_state = [noise*random.uniform(-1, 1)+next_state[i] for i in range(state_size)]
                #next_state = self.add_noise(next_state)

                sample = state + action + reward + next_state
                samples.append(sample)
                state = next_state
                
                sample_count = sample_count + 1
                episode_length += 1
                if done > 0.5 or done < -0.5:
                    break
                    
        df = pd.DataFrame(samples)
        df.columns = [f'variable_{i}' for i in range(state_size)] + ['action','reward'] + [f'nx_variable_{i}' for i in range(state_size)]
        self.df_data = df
        df.to_csv(filename, index=False)  

In [9]:
env_names = ['CartPole-v1', 'MountainCarContinuous-v0', 'MountainCar-v0', 'Pendulum-v1']
num_samples_list = ['10k', '20k', '30k', '40k', '50k']

for env_name in env_names:

    env = gym.make(env_name)
    batch_size = 32


    for num_samples in num_samples_list:
        
        #Initialize parameters for model
        action = env.action_space.sample()

        if isinstance(action, int):
            input_size = env.observation_space.shape[0] + 1
        elif isinstance(action, np.ndarray):
            input_size = env.observation_space.shape[0] + len(action)

        output_size = env.observation_space.shape[0] + 2
        num_hidden_layers = 3
        hidden_layer_nodes = 20
        activation = F.relu
        learning_rate = 0.01
        dropout_prob = 0.3
        num_networks = 1


        # Instantiate the model
        model = MC_Dropout_Net(input_size, output_size, num_hidden_layers, hidden_layer_nodes, activation, dropout_prob, num_networks).to(device)

        # Define loss function and optimizer
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        
        model_weights_file_path = 'montecarlodropout_'+ env_name + '_' + num_samples +'.pth'
#         model_weights_file_path = 'montecarlodropout_'+ env_name + '_' + num_samples + '_g=5' +'.pth'
        # Save the trained model
        model.load_state_dict(torch.load(model_weights_file_path))
#         torch.load(model.state_dict(), model_weights_file_path)
        
        sample = Sample(model,env)
        sample_filename = 'montecarlodropout_'+ env_name + '_' + num_samples + '_sample.csv'
#         sample_filename = 'montecarlodropout_'+ env_name + '_' + num_samples + '_g=5' + '_sample.csv'
        sample.generateSample(num_samples = 10000, noise = 0, filename = sample_filename)
        print(sample_filename)
        
#         break
#     break

montecarlodropout_CartPole-v1_10k_sample.csv
montecarlodropout_CartPole-v1_20k_sample.csv
montecarlodropout_CartPole-v1_30k_sample.csv
montecarlodropout_CartPole-v1_40k_sample.csv
montecarlodropout_CartPole-v1_50k_sample.csv
montecarlodropout_MountainCarContinuous-v0_10k_sample.csv
montecarlodropout_MountainCarContinuous-v0_20k_sample.csv
montecarlodropout_MountainCarContinuous-v0_30k_sample.csv
montecarlodropout_MountainCarContinuous-v0_40k_sample.csv
montecarlodropout_MountainCarContinuous-v0_50k_sample.csv
montecarlodropout_MountainCar-v0_10k_sample.csv
montecarlodropout_MountainCar-v0_20k_sample.csv
montecarlodropout_MountainCar-v0_30k_sample.csv
montecarlodropout_MountainCar-v0_40k_sample.csv
montecarlodropout_MountainCar-v0_50k_sample.csv
montecarlodropout_Pendulum-v1_10k_sample.csv
montecarlodropout_Pendulum-v1_20k_sample.csv
montecarlodropout_Pendulum-v1_30k_sample.csv
montecarlodropout_Pendulum-v1_40k_sample.csv
montecarlodropout_Pendulum-v1_50k_sample.csv


# Bayesian

In [9]:
import gym
!pip install torchbnn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchbnn as bnn

import sklearn
from sklearn.metrics import mean_squared_error, r2_score

import pickle

import os

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [10]:
class BayesianNN(nn.Module):
    def __init__(self, input_size, output_size, num_hidden_layers, hidden_layer_nodes, activation):
        super(BayesianNN, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.num_hidden_layers = num_hidden_layers
        self.hidden_layer_nodes = hidden_layer_nodes
        self.activation = activation
        
        # Define the layers
#         self.input_layer = nn.Linear(input_size, hidden_layer_nodes)
        self.input_layer = bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=input_size, out_features=hidden_layer_nodes)
        self.hidden_layers = nn.ModuleList()
        for _ in range(num_hidden_layers):
#           self.hidden_layers.append(nn.Linear(hidden_layer_nodes, hidden_layer_nodes))
            self.hidden_layers.append(bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=hidden_layer_nodes, out_features=hidden_layer_nodes))
#        self.output_layer = nn.Linear(hidden_layer_nodes, output_size)
        self.output_layer = bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=hidden_layer_nodes, out_features=output_size)
        

    def forward(self, x):
        x = self.activation(self.input_layer(x))
        for hidden_layer in self.hidden_layers:
            x = self.activation(hidden_layer(x))
        output = self.output_layer(x)
        return output


In [11]:
env_names = ['CartPole-v1', 'MountainCarContinuous-v0', 'MountainCar-v0', 'Pendulum-v1']
num_samples_list = ['10k', '20k', '30k', '40k', '50k']

for env_name in env_names:

    env = gym.make(env_name)
    batch_size = 32


    for num_samples in num_samples_list:
        
        #Initialize parameters for model
        action = env.action_space.sample()

        if isinstance(action, int):
            input_size = env.observation_space.shape[0] + 1
        elif isinstance(action, np.ndarray):
            input_size = env.observation_space.shape[0] + len(action)

        output_size = env.observation_space.shape[0] + 2
        num_hidden_layers = 3
        hidden_layer_nodes = 20
        activation = F.relu
        learning_rate = 0.01
        dropout_prob = 0.3
        num_networks = 5

        # Instantiate the model
        model = BayesianNN(input_size, output_size, num_hidden_layers, hidden_layer_nodes, activation).to(device)

        mse_loss = nn.MSELoss()
        kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
        kl_weight = 0.01

        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        
        model_weights_file_path = 'bayesian_'+ env_name + '_' + num_samples +'.pth' 
#         model_weights_file_path = 'bayesian_'+ env_name + '_' + num_samples + '_g=5' +'.pth'
        # Save the trained model
        model.load_state_dict(torch.load(model_weights_file_path))
#         torch.load(model.state_dict(), model_weights_file_path)
        
        sample = Sample(model,env)
        sample_filename = 'bayesian_'+ env_name + '_' + num_samples + '_sample.csv'
#         sample_filename = 'bayesian_'+ env_name + '_' + num_samples + '_g=5_sample.csv'
        sample.generateSample(num_samples = 10000, noise = 0, filename = sample_filename)
        print(sample_filename)
        
#         break
#     break

bayesian_CartPole-v1_10k_g=5_sample.csv
